In [1]:
import nltk

In [2]:
import jieba
import tflearn

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
import tensorflow as tf

# read the preprocressed corpus 

In [171]:
import pickle
with open('middleresult/tokenlizer_output_60000ch_60000en_40words_token.pkl','rb') as fhdl:
    (
         ind2ch,
         ch2ind,
         ind2en,
         en2ind,
         train_x,
         train_y,
    ) = pickle.load(fhdl)

In [172]:
len(ind2ch),len(ind2en)

(8823, 60000)

# define the model hyperparams

In [173]:
src_vocab_size = 50003#len(ind2en) + 3
target_vocat_size = 8826#len(ind2ch) + 3
attention_hidden_size = 512
attention_output_size = 512
embedding_size = 512
seq_max_len = 40
num_units = 512
batch_size = 64
layer_number = 2
max_grad = 1.0
dropout = 0.2

In [174]:
len(train_x),len(train_y)

(1925196, 1925196)

# process the model input

In [176]:
train_x = tf.contrib.keras.preprocessing.sequence.pad_sequences(train_x,seq_max_len,padding='post')
train_y = tf.contrib.keras.preprocessing.sequence.pad_sequences(train_y,seq_max_len,padding='post')

In [177]:
import random
index = random.randint(0,len(train_x))
print(' '.join([ind2en.get(i,'') for i in train_x[index]]))
print(' '.join([ind2ch.get(i,'') for i in train_y[index]]))

who ' ve lived there with their wonderful way of living                             
他 们 原 本 上 百 年 来 在 森 林 里 和 谐 安 居                        


In [178]:
train_x.shape,train_y.shape

((1925196, 40), (1925196, 40))

In [179]:
from sklearn.model_selection import train_test_split

In [180]:
train_x,test_x,train_y,test_y = train_test_split(train_x,train_y , test_size=0.01, random_state=42)

In [181]:
train_x[train_x >= src_vocab_size] = 1 
test_x[test_x >= src_vocab_size] = 1 

In [182]:
train_y[train_y >= target_vocat_size] = 1 
test_y[test_y >= target_vocat_size] = 1 

In [183]:
import numpy as np
x_last_index = np.max(train_x)
y_last_index = np.max(train_y)

ind2en[x_last_index] = '<go>'
ind2ch[y_last_index] = '<go>'
en2ind['<go>'] = x_last_index
ch2ind['<go>'] = y_last_index

ind2en[1] = '<unk>'
ind2ch[1] = '<unk>'
en2ind['<unk>'] = 1
ch2ind['<unk>'] = 1

In [184]:
import random
for i in range(5):
    index = random.randint(0,len(train_x))
    print(' '.join([ind2en.get(i,'') for i in train_x[index]]))
    print(' '.join([ind2ch.get(i,'') for i in train_y[index]]))

licence authorizing the use of a factory situated in hong kong for treating whales                          
批 准 在 位 於 香 港 的 工 厂 加 工 处 理 鲸 的 牌 照                      
here ' s what ' s different about people . we have the same needs ,                        
这 就 是 人 们 不 一 样 的 地 方 。 我 们 有 同 样 的 需 求 ，                   
this said , ... he wished to have me in his sight / once , as a friend : this fixed a day in spring               
我 膝 上 . 这 封 说 : 他 多 盼 望 有 个 机 会 , / 能 作 为 朋 友 , 见 一 见 我 . 这 一 封 又 订 了     
the issue occurs when authenticated mysql users overwrite arbitrary files by using a <unk> attack .                        
题 存 在 于 经 过 身 份 验 证 的 m y s q l 用 户 利 用 s y m l i n k 攻 击 覆 盖 任 意 文 件 的 过 程 中 。
watch your back , fish , ' cause squirrel master ain ' t gonna be there for you all the time .                  
小 心 点 ， 鱼 ， 斯 葵 尔 玛 斯 特 不 会 总 照 着 你 的 。                    


# define the translate nmt model

In [26]:
from tensorflow.python.layers import core as layers_core


In [27]:
import tensorflow as tf
import tflearn
tf.reset_default_graph()
config = tf.ConfigProto(log_device_placement=True,allow_soft_placement = True)
config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.Session(config=config)


with tf.device('/gpu:1'):
    initializer = tf.random_uniform_initializer(
        -0.08, 0.08)
    tf.get_variable_scope().set_initializer(initializer)
    
    x = tf.placeholder("int32", [None, None])
    y = tf.placeholder("int32", [None, None])
    y_in = tf.placeholder("int32",[None,None])
    x_len = tf.placeholder("int32",[None])
    y_len = tf.placeholder("int32",[None])
    x_real_len = tf.placeholder("int32",[None])
    y_real_len = tf.placeholder("int32",[None])
    learning_rate = tf.placeholder(tf.float32, shape=[])
    
    # embedding
    embedding_encoder = tf.get_variable(
        "embedding_encoder", [src_vocab_size, embedding_size],dtype=tf.float32)
    embedding_decoder = tf.get_variable(
        "embedding_decoder", [target_vocat_size, embedding_size],dtype=tf.float32)
    
    encoder_emb_inp = tf.nn.embedding_lookup(
        embedding_encoder, x)
    decoder_emb_inp = tf.nn.embedding_lookup(
        embedding_decoder, y_in)
    
    # encoder
    num_bi_layers = int(layer_number / 2)
    cell_list = []
    for i in range(num_bi_layers):
        cell_list.append(
            tf.contrib.rnn.DropoutWrapper(
                tf.contrib.rnn.BasicLSTMCell(num_units), input_keep_prob=(1.0 - dropout)
            )
        )
    if len(cell_list) == 1:
        encoder_cell = cell_list[0]
    else:
        encoder_cell = tf.contrib.rnn.MultiRNNCell(cell_list)
        
    cell_list = []
    
    for i in range(num_bi_layers):
        cell_list.append(
            tf.contrib.rnn.DropoutWrapper(
                tf.contrib.rnn.BasicLSTMCell(num_units), input_keep_prob=(1.0 - dropout)
            )
        )
    if len(cell_list) == 1:
        encoder_backword_cell = cell_list[0]
    else:
        encoder_backword_cell = tf.contrib.rnn.MultiRNNCell(cell_list)
    
    bi_outputs, bi_encoder_state = tf.nn.bidirectional_dynamic_rnn(
        encoder_cell,encoder_backword_cell, encoder_emb_inp,
        sequence_length=x_len,dtype=tf.float32)
    encoder_outputs = tf.concat(bi_outputs, -1)
    
    if num_bi_layers == 1:
        encoder_state = bi_encoder_state
    else:
        encoder_state = []
        for layer_id in range(num_bi_layers):
            encoder_state.append(bi_encoder_state[0][layer_id])  # forward
            encoder_state.append(bi_encoder_state[1][layer_id])  # backward
        encoder_state = tuple(encoder_state)
    
    # decoder 
    #decoder_cell = tf.contrib.rnn.BasicLSTMCell(num_units)
    cell_list = []
    for i in range(layer_number):
        cell_list.append(
            tf.contrib.rnn.DropoutWrapper(
                tf.contrib.rnn.BasicLSTMCell(num_units), input_keep_prob=(1.0 - dropout)
            )
        )
    if len(cell_list) == 1:
        decoder_cell = cell_list[0]
    else:
        decoder_cell = tf.contrib.rnn.MultiRNNCell(cell_list)
    
    # Helper
    
    # attention
    attention_mechanism = tf.contrib.seq2seq.LuongAttention(
        attention_hidden_size, encoder_outputs,
        memory_sequence_length=x_real_len,scale=True)
    decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
        decoder_cell, attention_mechanism,
        attention_layer_size=attention_output_size)
    
    
    projection_layer = layers_core.Dense(
        target_vocat_size, use_bias=False)
    
    
    
    # Dynamic decoding
    with tf.variable_scope("decode_layer"):
        helper = tf.contrib.seq2seq.TrainingHelper(
            decoder_emb_inp,sequence_length= y_len)
        decoder = tf.contrib.seq2seq.BasicDecoder(
            decoder_cell, helper, initial_state = decoder_cell.zero_state(dtype=tf.float32,batch_size=batch_size),
            output_layer=projection_layer)
       
        outputs, _,___  = tf.contrib.seq2seq.dynamic_decode(decoder)
        logits = outputs.rnn_output

        target_weights = tf.sequence_mask(
            y_real_len, seq_max_len, dtype=logits.dtype)
    
    # predicting
    # Helper
    with tf.variable_scope("decode_layer", reuse=True):
        helper_predict = tf.contrib.seq2seq.GreedyEmbeddingHelper(
            embedding_decoder,
            tf.fill([batch_size], ch2ind['<go>']), 0)
        decoder_predict = tf.contrib.seq2seq.BasicDecoder(
            decoder_cell, helper_predict, initial_state = decoder_cell.zero_state(dtype=tf.float32,batch_size=batch_size),
            output_layer=projection_layer)
        outputs_predict,_, __ = tf.contrib.seq2seq.dynamic_decode(
            decoder_predict, maximum_iterations=test_y.shape[1] * 2)
    translations = outputs_predict.sample_id

    # calculate loss
    crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=y, logits=logits)
    train_loss = (tf.reduce_sum(crossent * target_weights) /
        batch_size)
    
    optimizer_ori = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    trainable_params = tf.trainable_variables()
    gradients = tf.gradients(train_loss, trainable_params)
    clip_gradients, _ = tf.clip_by_global_norm(gradients, max_grad)
    global_step = tf.Variable(0, trainable=False, name='global_step')
    optimizer = optimizer_ori.apply_gradients(
            zip(clip_gradients, trainable_params), global_step=global_step)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(train_loss)
    #trainop = tflearn.TrainOp(loss=train_loss, optimizer=optimizer,
    #                          metric=train_loss, batch_size=64)


In [28]:
def cal_acc(logits,target):
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target[:,:seq_max_len], logits[:,:seq_max_len]))


# init the model

In [29]:
session.run(tf.global_variables_initializer())

In [30]:
saver = tf.train.Saver()

In [29]:
#saver.restore(session,'middleresult/align/result_1_20847')

INFO:tensorflow:Restoring parameters from middleresult/align/result_1_20847


In [32]:
saver.save(session,'middleresult/result_char')

'middleresult/result_char'

In [33]:
from utils import Dataset,ProgressBar

In [34]:
from utils import *
train_set = Dataset(train_x,train_y)
test_set = Dataset(test_x,test_y)

In [35]:
def get_bleu_score(predict,target):
    try:
        target = [[[j for index,j in enumerate(i) if j > 0 or index < 4]] for i in target]
        predict = [[j for index,j in enumerate(i) if j > 0 or index < 4] for i in predict]
        BLEUscore = nltk.translate.bleu_score.corpus_bleu(target,predict)
    except:
        BLEUscore = -1
    return BLEUscore

In [36]:
import numpy as np
def calc_test_loss(test_set = Dataset(test_x,test_y),display=True):
    accs = []
    worksum = int(len(test_x) / batch_size)
    loss_list = []
    predict_list = []
    target_list = []
    source_list = []
    pb = ProgressBar(worksum=worksum,info="validating...",auto_display=display)
    pb.startjob()
    #test_set = Dataset(test_x,test_y)
    for j in range(worksum):
        batch_x,batch_y = test_set.next_batch(batch_size)
        lx = [seq_max_len] * batch_size
        ly = [seq_max_len] * batch_size
        bx = [np.sum(m > 0) for m in batch_x]
        by = [np.sum(m > 0) for m in batch_y]
        tmp_loss,tran = session.run([train_loss,translations],feed_dict={x:batch_x,y:batch_y,
                                                     y_in:
                                                     np.concatenate((
                                                     np.ones((batch_y.shape[0],1),dtype=np.int) * ch2ind['<go>'],batch_y[:,:-1]) ,axis=1)
                                                     ,x_len:lx,y_len:ly,
                                                                        y_real_len:by,
                                                                        x_real_len:bx})
        loss_list.append(tmp_loss)
        tmp_acc = cal_acc(tran,batch_y)
        accs.append(tmp_acc)
        predict_list += [i for i in tran]
        target_list += [i for i in batch_y]
        source_list += [i for i in batch_x]
        pb.complete(1)
    return np.average(loss_list),np.average(accs),get_bleu_score(predict_list,target_list),predict_list,target_list,source_list

# calculate the initial loss and see what model outputs in the begining.

In [38]:
w_loss,w_acc,bleu_score,predict_list,target_list,source_list = calc_test_loss(Dataset(train_x[::100],train_y[::100]))

validating... 100.00 % [==================================================>] 300/300 	 used:120s eta:0 s

In [39]:
w_loss,w_acc,bleu_score

(233.5325, 5.5989583333333333e-05, 0.004130308764851706)

In [40]:
def get_all_text(x):
    return [' '.join([ind2ch.get(j,'') for j in i]) for i in x]
def get_all_en_text(x):
    return [' '.join([ind2en.get(j,'') for j in i]) for i in x]

In [41]:
texts = get_all_text(predict_list)
texts[:10]

['紮 紮 紮 紮 紮 麈 紮 麈 麈 麈 麈 麈 麈 麈 麈 黉 黉 佗 佗 黉 黉 黉 眃 眃 眃 眃 眃 眃 眃 眃 眃 格 格 格 格 贤 贤 格 ㏑ 格 格 格 格 格 ㏑ 格 格 ㏑ ㏑ ㏑ ㏑ ㏑ ㏑ ㏑ ㏑ ㏑ ㏑ ㏑ ㏑ 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 挪 衲 衲 衲 衲',
 '間 間 間 間 煜 煜 煜 臆 前 前 前 蝚 蝚 靘 μ ； 讫 讫 讫 讫 讫 讫 讫 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 鞑 蝚 铲 蝚 蝚 蝚 铲 铲 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 铲 铲 铲 铲 铲 铲 铲 铲 睦 睦 睦 睦 睦 睦 睦 睦 睦 睦 睦 睦 睦 睦 睦 睦',
 '馓 戴 籁 庥 龋 龋 语 д ４ 囊 砖 气 气 气 气 气 气 气 气 气 气 气 气 气 气 粕 民 民 民 民 民 民 赔 民 筽 民 迹 迹 迹 迹 迹 迹 迹 鰶 犸 犸 犸 犸 犸 天 天 天 天 猴 猴 炫 猴 猴 洐 洐 猴 腋 钩 钩 钩 钩 钩 马 \ue009 臻 臻 臻 臻 \ue009 \ue009 \ue009 \ue009 \ue009 \ue009 \ue009',
 '呔 呔 浼 浼 浼 浼 浼 浼 驶 里 里 里 里 虬 蝚 蝚 灑 灑 蝚 蝚 蝚 蝚 》 苞 苞 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 涩 钭 钭 钭 钭 钭 钭 钭 钭 鯦 鯦 钭 骄 钭 钭 钭 桕 桕 桕 未 桕 桕 桕 拾 拾 拾 拾 拾 绗 绗 鰶',
 '庢 庢 庢 臆 臆 臆 臆 臆 磌 磌 磌 磌 磌 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 衲 \ue40f 奇 \ue40f 衲 衲 叻 奇 奇 圈 圈 谜 谜 谜 谜 谜 谜 谜 谜 谜 谜 谜 谜 谜 谜 谜 谜 谜 辆 辆 谜 辆 辆 辆 辆 辆 辆 罙 阍 罙 阍 阍 阍 叻 阍 阍 叻',
 '轰 轰 轰 浼 癞 浼 臆 臆 臆 臆 臆 臆 臆 臆 臆 臆 铜 臆 臆 臆 臆 臆 灑 灑 灑 灑 冼 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 蝚 鞑 蝚 鞑 鞑 鞑 蝚 

In [42]:
texts = get_all_text(target_list)
texts[:10]

['不 然 的 话 ， 我 把 你 屁 股 捣 烂 - - 噢 ， 上 帝 啊                     ',
 '医 生 给 她 注 射 以 减 轻 疼 痛                             ',
 '“ 我 们 希 望 中 国 能 够 慎 重 考 虑 很 多 谈 判 伙 伴 本 周 表 达 的 关 切 ， 调 整 立 场 ， 及 早 恢 复 谈 判 。 ”',
 '《 牛 津 当 代 大 辞 典 》 词 库 1 0 0 % 采 用 真 人 女 声 。                  ',
 '定 一 个 牌 手 基 于 对 手 给 他 的 错 误 信 息 采 取 了 行 动 ， 将 酌 情 按 第 2 1 条 或 第 4 7 条 e 款 处 理 。',
 '易 腐 产 品 标 准 化 和 质 量 改 进 工 作 队                         ',
 '本 品 具 有 优 越 的 匀 染 效 果 ， 优 越 的 耐 硬 水 及 耐 盐 性 。                 ',
 '由 通 知 行 签 发 的 本 信 用 证 正 本 通 知 书 ， 因 为 有 效 信 用 证 必 须 由 指 定 银 行 在 该 通 知 书 上 背 书 。',
 'r s s 根 据 应 用 的 地 址 将 中 断 指 向 特 定 的 处 理 器 内 核 。                ',
 '） 报 送 上 市 公 司 收 购 报 告 书 时 所 持 有 被 收 购 公 司 股 份 数 占 该 公 司 已 发 行 的 股 份 总 数 的 比 例 。']

# now train the model

In [43]:
#tran.shape
i_save = 0
j_save = 0

In [44]:
print(i_save,j_save)

0 0


In [45]:
model_path = 'align_char'

In [46]:
os.mkdir('middleresult/{}'.format(model_path))
os.mkdir('eval/{}'.format(model_path))

In [49]:
n_epoch = 15
restore = True
#lr = 1
for i in range(i_save,n_epoch):
    
    i_save = i
    worksum = int(len(train_y)/batch_size)
    pb = ProgressBar(worksum=worksum)
    pb.startjob()
    train_loss_list = []
    train_acc_list = []
    for j in range(worksum):
        if restore == True and j < j_save:
            pb.finishsum += 1
            continue
        restore = False
        
        j_save = j
        batch_x,batch_y = train_set.next_batch(batch_size)
        lx = [seq_max_len] * batch_size
        ly = [seq_max_len] * batch_size
        bx = [np.sum(m > 0) for m in batch_x]
        by = [np.sum(m > 0) for m in batch_y]
        by =[m + 2  if m < seq_max_len - 1 else m for m in by ]
        _, loss = session.run([optimizer,train_loss],feed_dict={x:batch_x,y:batch_y,x_len:lx,y_len:ly,learning_rate:lr,y_in:
                                                                np.concatenate((
                                                                np.ones((batch_y.shape[0],1),dtype=np.int) * ch2ind['<go>'],batch_y[:,:-1]) ,axis=1)
                                                                ,y_real_len:by,
                                                                x_real_len:bx
                                                               })
        train_loss_list.append(loss)
        #tmp_train_acc = cal_acc(tran,batch_y)
        #train_acc_list.append(tmp_train_acc)
        pb.info = "iter {} loss:{} lr:{}".format(i + 1,loss,lr)
        val_step = int(worksum / 4)
        if j % val_step == 0 and j != 0:
            test_loss,test_acc,bleu_score,predict_list,target_list,source_list = calc_test_loss()
            _,train_acc,train_bleu_score,train_predict_list,train_target_list,train_source_list = calc_test_loss(Dataset(train_x[::100],train_y[::100]),display=False)
            predict_texts = get_all_text(predict_list)
            target_texts = get_all_text(target_list)
            source_texts = get_all_en_text(source_list)
            
            train_predict_texts = get_all_text(train_predict_list)
            train_target_texts = get_all_text(train_target_list)
            train_source_texts = get_all_en_text(train_source_list)
            
            with open('eval/{}/{}_{}_predict'.format(model_path,i + 1,j),'w',encoding='utf-8') as whdl:
                for line in predict_texts:
                    whdl.write("{}\n".format(line))
            with open('eval/{}/{}_{}_target'.format(model_path,i + 1,j),'w',encoding='utf-8') as whdl:
                for line in target_texts:
                    whdl.write("{}\n".format(line))
            with open('eval/{}/{}_{}_source'.format(model_path,i + 1,j),'w',encoding='utf-8') as whdl:
                for line in source_texts:
                    whdl.write("{}\n".format(line))
                    
            with open('eval/{}/{}_{}_predict_train'.format(model_path,i + 1,j),'w',encoding='utf-8') as whdl:
                for line in train_predict_texts:
                    whdl.write("{}\n".format(line))
            with open('eval/{}/{}_{}_target_train'.format(model_path,i + 1,j),'w',encoding='utf-8') as whdl:
                for line in train_target_texts:
                    whdl.write("{}\n".format(line))
            with open('eval/{}/{}_{}_source_train'.format(model_path,i + 1,j),'w',encoding='utf-8') as whdl:
                for line in train_source_texts:
                    whdl.write("{}\n".format(line))
            print("\niter {} step {} train loss {} train acc {} test loss {} test acc {} bleu {} lr {}\n".format(i+1,j,np.average(train_loss_list[-val_step:]),train_acc,test_loss,test_acc,bleu_score,lr))
            try:
                saver = tf.train.Saver()
                saver.save(session,'middleresult/{}/result_{}_{}'.format(model_path,i + 1,j))
            except:
                print('save fail')
        lr_step = int(worksum / 2) - 1
        if j % lr_step == 0 and j != 0:
            if (i + 1) > 10:
                lr = lr / 2
        pb.complete(1)

validating... 100.00 % [==================================================>] 300/300 	 used:106s eta:0 s7445/29780 	 used:1984s eta:5951 s
iter 13 step 7445 train loss 51.49441909790039 train acc 0.3890546875 test loss 56.52604675292969 test acc 0.3802877604166666 bleu 0.20683722390776155 lr 0.0625

validating... 100.00 % [==================================================>] 300/300 	 used:105s eta:0 s14890/29780 	 used:4461s eta:4460 s
iter 13 step 14890 train loss 51.49875259399414 train acc 0.38941145833333335 test loss 56.567684173583984 test acc 0.38301171875000006 bleu 0.2169776407879747 lr 0.03125

validating... 100.00 % [==================================================>] 300/300 	 used:103s eta:0 s22335/29780 	 used:6938s eta:2312 ss
iter 13 step 22335 train loss 51.54439163208008 train acc 0.39030338541666665 test loss 56.541290283203125 test acc 0.3805 bleu 0.22007911262894841 lr 0.03125

validating... 100.00 % [==================================================>] 300/300 	

KeyboardInterrupt: 

In [50]:
session

In [51]:
len([ind2en.get(i,'') for i in test_x[1]])

40

# try to translate

In [227]:
sent = 'you are too stupid to know that you are an idiot .'.split()

In [228]:
sent

['you',
 'are',
 'too',
 'stupid',
 'to',
 'know',
 'that',
 'you',
 'are',
 'an',
 'idiot',
 '.']

In [229]:
sents = [en2ind.get(i) for i in sent]

In [230]:
sents = tf.contrib.keras.preprocessing.sequence.pad_sequences([sents],seq_max_len,padding='post')

In [231]:
sents

array([[  13,   31,  198, 3197,    7,   83,   17,   13,   31,   35, 7122,
           3,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0]])

In [232]:
np.repeat(sents,35,axis=0)

array([[ 13,  31, 198, ...,   0,   0,   0],
       [ 13,  31, 198, ...,   0,   0,   0],
       [ 13,  31, 198, ...,   0,   0,   0],
       ..., 
       [ 13,  31, 198, ...,   0,   0,   0],
       [ 13,  31, 198, ...,   0,   0,   0],
       [ 13,  31, 198, ...,   0,   0,   0]])

In [233]:
sum(sents[0] > 0) + 1

13

In [234]:
tran = session.run([translations],feed_dict={x:np.repeat(sents,64,axis=0),x_len:[35] * 64, x_real_len:[sum(sents[0] > 0) + 1] * 64})

In [235]:
from collections import Counter
Counter(''.join([ind2ch.get(i,'') for i in j]) for j in tran[0]).most_common(5)

[('你太傻了，知道你是个白痴', 34),
 ('你太傻了，不知道你是个白痴', 8),
 ('你太傻了，你就知道你是个白痴', 6),
 ('你太愚蠢了，知道你是个白痴', 2),
 ('你太傻了，知道你是白痴', 2)]

In [203]:
en2ind['<go>'],ch2ind['<go>']

(50002, 8824)

# release the model

In [135]:
!mkdir release

A subdirectory or file release already exists.


In [136]:
os.mkdir('release/align_and_translate_char_50000')

In [137]:
saver.save(session,'release/align_and_translate_char_50000/align_and_translate_model')

'release/align_and_translate_char_50000/align_and_translate_model'

In [207]:
with open('release/align_and_translate_char_50000/dic.pkl','wb') as whdl:
    pickle.dump(
        ( 
            ind2ch,
            ch2ind,
            ind2en,
            en2ind,
        ),whdl,protocol=2)